# Moje narzędzia #1 - wstępna obróbka zbioru - Kwiecień 2020

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df= pd.read_csv('/home/wojciech/Pulpit/4/kliki_uczenie.csv', sep=';')
print(df.shape)
print()
print(df.columns)
df.head(3)

(3619621, 24)

Index(['id', 'klik', 'data_godzina', 'baner_pozycja', 'strona_id',
       'strona_domena', 'strona_kategoria', 'aplikacja_id', 'aplikacja_domena',
       'aplikacja_kategoria', 'urz_id', 'urz_ip', 'urz_model', 'urz_typ',
       'urz_polaczenie', 'kat1', 'kat2', 'kat3', 'kat4', 'kat5', 'kat6',
       'kat7', 'kat8', 'kat9'],
      dtype='object')


,id,klik,data_godzina,baner_pozycja,strona_id,strona_domena,strona_kategoria,aplikacja_id,aplikacja_domena,aplikacja_kategoria,...,urz_polaczenie,kat1,kat2,kat3,kat4,kat5,kat6,kat7,kat8,kat9
0,93718246913880603,0,17030100,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,...,0,105,119743,10320,10050,12264,13,10427,100000,61
1,93718246913880604,0,17030100,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,105,115706,10320,10050,11722,10,10035,-1,79
2,93718246913880605,0,17030100,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,...,0,105,120352,10320,10050,12333,10,10039,-1,157


## <span style="color:green"> Wielkość bazy df jest ogromna - wyciągam tylko 10% losowo wybranych rekordów</span> 
Taka metoda jest najbardziej reprezentatywna i oszczędza zasoby komputera.

In [2]:
df2 = df.sample(frac = 0.1, random_state=10)
df2.shape

(361962, 24)

## <span style="color:green"> Wyświetlenie braków </span>  (własnej roboty)

In [3]:
a,b = df2.shape     #<- ile mamy kolumn
b

print('NUMBER OF EMPTY RECORDS vs. FULL RECORDS')
print('----------------------------------------')
for i in range(1,b):
    i = df.columns[i]
    r = df[i].isnull().sum()
    h = df[i].count()
   
    if r > 0:
        print(i,"--------",r,"--------",h) 

NUMBER OF EMPTY RECORDS vs. FULL RECORDS
----------------------------------------


## <span style="color:green"> Nie warto kasować rekordów z brakami tylko warto wstawiać tam wartości z poza zakresu </span>

Wyszukuje tylko puste rekordy

In [4]:
null_value_stats = df2.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

Wstawia w puste miejsca wartości z poza zakresu

In [5]:
df2.fillna(-777, inplace=True)

## <span style="color:green"> Przegląd danych pod kontem formatu i ilości wartości unikalnych</span>  (własnej roboty)

In [6]:
a,b = df2.shape     #<- ile mamy kolumn
b

import numpy as np

for i in range(1,b):
    i = df2.columns[i]
    h = df2[i].nunique()
    f = df2[i].dtypes
          
    print(f,"---",h,"---", i)

int64 --- 2 --- klik
int64 --- 216 --- data_godzina
int64 --- 7 --- baner_pozycja
object --- 2096 --- strona_id
object --- 2086 --- strona_domena
object --- 21 --- strona_kategoria
object --- 2206 --- aplikacja_id
object --- 137 --- aplikacja_domena
object --- 25 --- aplikacja_kategoria
object --- 59172 --- urz_id
object --- 236559 --- urz_ip
object --- 4297 --- urz_model
int64 --- 4 --- urz_typ
int64 --- 4 --- urz_polaczenie
int64 --- 7 --- kat1
int64 --- 1897 --- kat2
int64 --- 8 --- kat3
int64 --- 9 --- kat4
int64 --- 389 --- kat5
int64 --- 4 --- kat6
int64 --- 64 --- kat7
int64 --- 158 --- kat8
int64 --- 55 --- kat9


## <span style="color:green"> Które z tych funkcji są kategoryczne?</span> (własnej roboty)

Stawiam dwa warunki: 

1. funkcja jest kategoryczna bo ma format 'object'
2. funkcja jest kategoryczna bo ma do 10 unikalnych wartości

In [7]:
a,b = df2.shape     #<- ile mamy kolumn
b

print('ONLY DISCRETE FUNCTION')
print('----------------------')
for i in range(1,b):
    i = df2.columns[i]
    f = df2[i].dtypes
    h = df2[i].nunique()
    
    if f == np.object or h<=10:
        print(i,"---",f,"---",h)

ONLY DISCRETE FUNCTION
----------------------
klik --- int64 --- 2
baner_pozycja --- int64 --- 7
strona_id --- object --- 2096
strona_domena --- object --- 2086
strona_kategoria --- object --- 21
aplikacja_id --- object --- 2206
aplikacja_domena --- object --- 137
aplikacja_kategoria --- object --- 25
urz_id --- object --- 59172
urz_ip --- object --- 236559
urz_model --- object --- 4297
urz_typ --- int64 --- 4
urz_polaczenie --- int64 --- 4
kat1 --- int64 --- 7
kat3 --- int64 --- 8
kat4 --- int64 --- 9
kat6 --- int64 --- 4


#### Potrzebujemy listy z numerami kolumnaby użyć jej w CatBoost¶

In [8]:
import numpy as np

categorical_fun = np.where((df2.dtypes == np.object)|(df2.nunique() <= 10))[0]
categorical_fun

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18, 20])

#### Wyświetlamy co to za kolumny

In [9]:
PPS = categorical_fun

KOT_MIC = dict(zip(df2, PPS))
KOT_sorted_keys_MIC = sorted(KOT_MIC, key=KOT_MIC.get, reverse=False)

for r in KOT_sorted_keys_MIC:
    print (r, KOT_MIC[r])

id 1
klik 3
data_godzina 4
baner_pozycja 5
strona_id 6
strona_domena 7
strona_kategoria 8
aplikacja_id 9
aplikacja_domena 10
aplikacja_kategoria 11
urz_id 12
urz_ip 13
urz_model 14
urz_typ 15
urz_polaczenie 17
kat1 18
kat2 20


## <span style="color:red"> Uwaga! 
#### <span style="color:red">    1. dane categorialne nie muszą mieć tylko formatu tekstowego, 

#### <span style="color:red">    2. dane tekstowe trzeba przerobić do formatu numerycznego 

### <span style="color:green"> Narzędzie do automatycznego kodowania zmiennych dyskretnych </span>(własnej roboty)


In [10]:
a,b = df2.shape     #<- ile mamy kolumn
b

print('DISCRETE FUNCTIONS CODED')
print('------------------------')
for i in range(1,b):
    i = df2.columns[i]
    f = df2[i].dtypes
    if f == np.object:
        print(i,"---",f)   
    
        if f == np.object:
        
            df2[i] = pd.Categorical(df2[i]).codes
        
            continue

DISCRETE FUNCTIONS CODED
------------------------
strona_id --- object
strona_domena --- object
strona_kategoria --- object
aplikacja_id --- object
aplikacja_domena --- object
aplikacja_kategoria --- object
urz_id --- object
urz_ip --- object
urz_model --- object
